In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig
import os
import gc
import re

# 配置设置
base_model_name = "unsloth/Qwen3-14B"
adapter_path = ""  # 适配器目录
csv_path = r""  # 替换为您的CSV文件路径
output_csv_path = r""  # 替换为输出文件路径
batch_size = 1  # 批处理大小


# 1. 加载模型和分词器
def load_model():
    print("⏳ 加载基础模型...")
    # 先加载基础模型
    base_model, _ = FastLanguageModel.from_pretrained(
        model_name=base_model_name,
        max_seq_length=2048,
        load_in_4bit=True,
        device_map="auto"
    )
    
    print("🔄 加载PEFT适配器...")
    # 使用标准的PeftModel加载适配器
    model = PeftModel.from_pretrained(
        base_model,
        adapter_path,
        adapter_name="accident_cause_adapter"
    )
    model.eval()
    
    print("🔠 加载分词器...")
    # 从适配器目录加载分词器
    tokenizer = AutoTokenizer.from_pretrained(
        adapter_path,
        padding_side="left",
        truncation_side="left"
    )
    tokenizer.pad_token = tokenizer.eos_token
    
    return model, tokenizer

# 2. 创建推理管道 - 修正设备问题
def create_inference_pipeline(model, tokenizer):
    print("⚙️ 创建推理管道...")
    # 移除device参数，因为模型已通过device_map自动分配到设备
    return pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
    )

# 3. 生成响应的函数
def generate_response(pipe, input_text):
    try:
        # 构建Alpaca格式提示
        prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
你是一个事故因果推理专家，需要根据input中的事故过程从给出的直接原因和间接原因分类表中推理出事故对应的直接原因和间接原因，输出只能为：思维链。直接原因:直接原因列表，间接原因:间接原因列表。直接原因分类表有:车辆操作安全意识淡薄,超速与载重合规性,高风险路段驾驶行为不合规,重型车辆操作规范,车辆行驶稳定性管理,车间装卸货物规范安全操作,工地卸载货物操作不规范.间接原因分类表有:施工现场人员安全管理体系不健全,运输过程合规性监管失效,重型车辆驾驶员运输培训不到位,企业管理车辆驾驶规范责任失效。

### Input:
{}

### Response:
"""
        formatted_prompt = prompt_template.format(input_text)
        
        # 生成参数
        generation_config = {
            "max_new_tokens":2048,  # 减少生成长度以节省内存
            "temperature": 0.6,    # 降低随机性
            "top_p": 0.9,
            "top_k": 30,
            "do_sample": True,
            "pad_token_id": pipe.tokenizer.eos_token_id,
            "repetition_penalty": 1.2
        }
        
        # 生成响应
        outputs = pipe(
            formatted_prompt,
            **generation_config,
            return_full_text=False,
            batch_size=batch_size
        )
        
        # 提取响应内容
        response = outputs[0]['generated_text'].strip()
        
        # 清理响应内容
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()
        
        # 移除可能的重复内容
        response = re.sub(r'(直接原因|间接原因).*?[:：]', '', response, count=1)
        
        return response
    
    except Exception as e:
        print(f"❌ 生成错误: {str(e)}")
        return f"ERROR: {str(e)}"

# 4. 主处理流程
def process_csv(csv_path, output_path):
    # 加载模型
    model, tokenizer = load_model()
    pipe = create_inference_pipeline(model, tokenizer)
    
    # 读取CSV文件
    print(f"📖 读取CSV文件: {csv_path}")
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"❌ 读取CSV失败: {str(e)}")
        return
    
    # 确保存在目标列
    if "事故中断过程描述" not in df.columns:
        print("❌ CSV文件中缺少'事故中断过程描述'列")
        return
    
    # 添加结果列
    df["模型推理结果"] = ""
    processed_count = 0
    
    print("🚀 开始处理事故描述...")
    total_rows = len(df)
    
    # 使用tqdm创建进度条
    progress_bar = tqdm(total=total_rows, desc="处理事故数据", unit="行")
    
    for i in range(total_rows):
        accident_desc = df.at[i, "事故中断过程描述"]
        
        # 跳过空描述
        if pd.isna(accident_desc) or str(accident_desc).strip() == "":
            df.at[i, "模型推理结果"] = "SKIPPED: 空描述"
            progress_bar.update(1)
            continue
            
        try:
            # 生成推理结果
            result = generate_response(pipe, str(accident_desc))
            df.at[i, "模型推理结果"] = result
            processed_count += 1
            
            # 每处理5行保存一次中间结果
            if processed_count > 0 and processed_count % 5 == 0:
                df.to_csv(output_path, index=False, encoding='utf-8-sig')
                print(f"💾 已保存中间结果 ({processed_count}/{total_rows})")
            
            # 每处理10行清理一次内存
            if processed_count > 0 and processed_count % 10 == 0:
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                gc.collect()
                print("🧹 已清理GPU缓存")
                
        except Exception as e:
            print(f"❌ 行 {i+1} 处理失败: {str(e)}")
            df.at[i, "模型推理结果"] = f"ERROR: {str(e)}"
        
        progress_bar.update(1)
    
    progress_bar.close()
    
    # 最终保存
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✅ 处理完成! 共处理 {processed_count} 条记录，结果保存至: {output_path}")
    
    # 释放资源
    del model, tokenizer, pipe
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# 执行处理
if __name__ == "__main__":
    # 检查路径是否存在
    if not os.path.exists(adapter_path):
        print(f"❌ 适配器路径不存在: {adapter_path}")
    else:
        print(f"🔍 适配器路径验证: {adapter_path} (包含 {len(os.listdir(adapter_path))} 个文件)")
    
    # 设置环境变量防止并行处理冲突
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    
    # 设置CUDA设备可见性
    if torch.cuda.is_available():
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 指定使用第一块GPU
    
    process_csv(csv_path, output_csv_path)


In [ ]:
import pandas as pd
import json
import re
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# 定义原因分类表
DIRECT_CAUSES = [
    "车辆操作安全意识淡薄",
    "超速与载重合规性",
    "高风险路段驾驶行为不合规",
    "重型车辆操作规范",
    "车辆行驶稳定性管理",
    "车间装卸货物规范安全操作",
    "工地卸载货物操作不规范"
]

INDIRECT_CAUSES = [
    "施工现场人员安全管理体系不健全",
    "运输过程合规性监管失效",
    "重型车辆驾驶员运输培训不到位",
    "企业管理车辆驾驶规范责任失效"
]

class RobustCauseParser:
    """鲁棒的原因解析器，处理不同格式的原因描述"""
    
    def extract_causes(self, text):
        """从文本中提取原因列表"""
        if pd.isna(text) or not text:
            return [], []
            
        # 尝试从文本中提取原因
        direct_list = []
        indirect_list = []
        
        # 尝试匹配格式化的原因列表
        if "直接原因:" in text and "间接原因:" in text:
            try:
                # 尝试提取直接原因部分
                direct_part = re.search(r"直接原因[:：]\s*(.+?)(间接原因|$)", text, re.DOTALL)
                if direct_part:
                    direct_str = direct_part.group(1).strip()
                    direct_list = [c.strip() for c in re.split(r"[,，]", direct_str) if c.strip()]
                
                # 尝试提取间接原因部分
                indirect_part = re.search(r"间接原因[:：]\s*(.+)", text)
                if indirect_part:
                    indirect_str = indirect_part.group(1).strip()
                    indirect_list = [c.strip() for c in re.split(r"[,，]", indirect_str) if c.strip()]
            except:
                pass
        
        # 如果没有匹配到格式化的原因，尝试从自由文本中匹配
        if not direct_list:
            for cause in DIRECT_CAUSES:
                if cause in text:
                    direct_list.append(cause)
        
        if not indirect_list:
            for cause in INDIRECT_CAUSES:
                if cause in text:
                    indirect_list.append(cause)
        
        return direct_list, indirect_list

class CauseMapper:
    """将原始原因映射到标准分类表"""
    
    def __init__(self, direct_causes, indirect_causes):
        self.direct_causes = direct_causes
        self.indirect_causes = indirect_causes
        
    def map_direct(self, causes):
        """将原始直接原因映射到标准分类"""
        mapped = []
        for cause in causes:
            # 尝试完全匹配
            if cause in self.direct_causes:
                mapped.append(cause)
                continue
                
            # 尝试部分匹配
            for standard_cause in self.direct_causes:
                if standard_cause in cause or cause in standard_cause:
                    mapped.append(standard_cause)
                    break
        return list(set(mapped))  # 去重
    
    def map_indirect(self, causes):
        """将原始间接原因映射到标准分类"""
        mapped = []
        for cause in causes:
            if cause in self.indirect_causes:
                mapped.append(cause)
                continue
                
            for standard_cause in self.indirect_causes:
                if standard_cause in cause or cause in standard_cause:
                    mapped.append(standard_cause)
                    break
        return list(set(mapped))

def calculate_classification_metrics(true_labels, pred_labels, class_list):
    """计算分类指标（宏平均）"""
    # 将标签列表转换为二进制向量
    true_vectors = []
    pred_vectors = []
    
    for true, pred in zip(true_labels, pred_labels):
        true_vec = [1 if c in true else 0 for c in class_list]
        pred_vec = [1 if c in pred else 0 for c in class_list]
        true_vectors.append(true_vec)
        pred_vectors.append(pred_vec)
    
    # 展平向量用于整体指标计算
    flat_true = np.array(true_vectors).flatten()
    flat_pred = np.array(pred_vectors).flatten()
    
    # 计算整体指标
    precision = precision_score(flat_true, flat_pred, average='macro', zero_division=0)
    recall = recall_score(flat_true, flat_pred, average='macro', zero_division=0)
    f1 = f1_score(flat_true, flat_pred, average='macro', zero_division=0)
    
    # 计算每个类别的指标
    class_metrics = {}
    for i, cause in enumerate(class_list):
        class_true = [vec[i] for vec in true_vectors]
        class_pred = [vec[i] for vec in pred_vectors]
        
        class_metrics[cause] = {
            "precision": precision_score(class_true, class_pred, zero_division=0),
            "recall": recall_score(class_true, class_pred, zero_division=0),
            "f1": f1_score(class_true, class_pred, zero_division=0),
            "support": sum(class_true)
        }
    
    return {
        "overall": {
            "precision": precision,
            "recall": recall,
            "f1": f1
        },
        "per_class": class_metrics
    }

def evaluate_csv(csv_path):
    """评估CSV文件中的预测结果"""
    # 读取CSV文件
    print(f"📖 读取CSV文件: {csv_path}")
    try:
        df = pd.read_csv(csv_path, encoding='utf-8-sig')
    except Exception as e:
        print(f"❌ 读取CSV失败: {str(e)}")
        return None
    
    # 检查必要列是否存在
    required_columns = ['直接原因', '间接原因', '匹配的直接原因', '匹配的间接原因']
    for col in required_columns:
        if col not in df.columns:
            print(f"❌ CSV文件中缺少'{col}'列")
            return None
    
    # 初始化工具
    parser = RobustCauseParser()
    mapper = CauseMapper(DIRECT_CAUSES, INDIRECT_CAUSES)
    
    # 准备评估数据容器
    evaluation_data = {
        "direct": {"true": [], "pred": []},
        "indirect": {"true": [], "pred": []},
        "details": []
    }
    
    print("🔍 解析原因并映射到分类表...")
    
    # 遍历每一行
    for i, row in tqdm(df.iterrows(), total=len(df), desc="处理行"):
        # 解析真实原因
        true_direct_raw = str(row["直接原因"]) if not pd.isna(row["直接原因"]) else ""
        true_indirect_raw = str(row["间接原因"]) if not pd.isna(row["间接原因"]) else ""
        
        # 解析预测原因
        pred_direct_raw = str(row["匹配的直接原因"]) if not pd.isna(row["匹配的直接原因"]) else ""
        pred_indirect_raw = str(row["匹配的间接原因"]) if not pd.isna(row["匹配的间接原因"]) else ""
        
        # 映射到分类表
        true_direct_mapped = mapper.map_direct(parser.extract_causes(true_direct_raw)[0])
        true_indirect_mapped = mapper.map_indirect(parser.extract_causes(true_indirect_raw)[1])
        pred_direct_mapped = mapper.map_direct(parser.extract_causes(pred_direct_raw)[0])
        pred_indirect_mapped = mapper.map_indirect(parser.extract_causes(pred_indirect_raw)[1])
        
        # 存储结果
        evaluation_data["direct"]["true"].append(true_direct_mapped)
        evaluation_data["direct"]["pred"].append(pred_direct_mapped)
        evaluation_data["indirect"]["true"].append(true_indirect_mapped)
        evaluation_data["indirect"]["pred"].append(pred_indirect_mapped)
        
        # 保存详细结果用于分析
        evaluation_data["details"].append({
            "index": i,
            "事故中断过程描述": row["事故中断过程描述"] if "事故中断过程描述" in row else "",
            "true_direct_raw": true_direct_raw,
            "true_indirect_raw": true_indirect_raw,
            "pred_direct_raw": pred_direct_raw,
            "pred_indirect_raw": pred_indirect_raw,
            "true_direct_mapped": true_direct_mapped,
            "true_indirect_mapped": true_indirect_mapped,
            "pred_direct_mapped": pred_direct_mapped,
            "pred_indirect_mapped": pred_indirect_mapped
        })
    
    # 计算分类指标
    print("📊 计算评估指标...")
    results = {
        "direct": calculate_classification_metrics(
            evaluation_data["direct"]["true"], 
            evaluation_data["direct"]["pred"], 
            DIRECT_CAUSES
        ),
        "indirect": calculate_classification_metrics(
            evaluation_data["indirect"]["true"], 
            evaluation_data["indirect"]["pred"], 
            INDIRECT_CAUSES
        )
    }
    
    # 打印结果
    print("\n===== 评估结果 =====")
    print(f"直接原因精确率: {results['direct']['overall']['precision']:.4f}")
    print(f"直接原因召回率: {results['direct']['overall']['recall']:.4f}")
    print(f"直接原因F1: {results['direct']['overall']['f1']:.4f}")
    print(f"间接原因精确率: {results['indirect']['overall']['precision']:.4f}")
    print(f"间接原因召回率: {results['indirect']['overall']['recall']:.4f}")
    print(f"间接原因F1: {results['indirect']['overall']['f1']:.4f}")
    
    # 保存完整结果
    output_path = csv_path.replace(".csv", "_evaluation_results.json")
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump({
            "metrics": results,
            "details": evaluation_data["details"]
        }, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ 评估完成! 详细结果已保存至: {output_path}")
    
    return results

# 执行评估
if __name__ == "__main__":
    # CSV文件路径
    csv_path = r""
    
    # 运行评估
    results = evaluate_csv(csv_path)
    
    # 打印每个类别的详细指标
    if results:
        print("\n===== 直接原因详细指标 =====")
        for cause, metrics in results["direct"]["per_class"].items():
            print(f"{cause}:")
            print(f"  精确率: {metrics['precision']:.4f}")
            print(f"  召回率: {metrics['recall']:.4f}")
            print(f"  F1: {metrics['f1']:.4f}")
            print(f"  支持度: {metrics['support']}")
        
        print("\n===== 间接原因详细指标 =====")
        for cause, metrics in results["indirect"]["per_class"].items():
            print(f"{cause}:")
            print(f"  精确率: {metrics['precision']:.4f}")
            print(f"  召回率: {metrics['recall']:.4f}")
            print(f"  F1: {metrics['f1']:.4f}")
            print(f"  支持度: {metrics['support']}")
